# pytketを用いたQPE計算

この章では、pytketを用いたlong-termアルゴリズムの実装例を示します。

[pytket user guideのQPEの解説](https://docs.quantinuum.com/tket/user-guide/examples/algorithms_and_protocols/phase_estimation.html)

HamiltonianやVQEの章で作っておいた自作関数などを再び書き写しておいて...

In [2]:
#必要に応じて...
#!pip install pytket
#!pip install pytket-qiskit
#!pip install pytket-projectq
#!pip install openfermion
#!pip install seaborn

import numpy as np
import itertools
from itertools import combinations
import seaborn as sns
cols = sns.color_palette("deep")
import matplotlib.pyplot as plt
import pandas as pd
from openfermion import QubitOperator
from pytket.circuit import Circuit, Qubit, Bit
from pytket.circuit import PauliExpBox
from pytket.circuit.display import render_circuit_jupyter as draw
from pytket.utils import gate_counts
from pytket.utils import probs_from_state
from pytket.utils.expectations import get_operator_expectation_value
from pytket.extensions.qiskit import AerStateBackend
from pytket.extensions.qiskit import AerBackend
from pytket.pauli import Pauli, QubitPauliString
from pytket.utils.operators import QubitPauliOperator
from scipy.optimize import minimize_scalar

class PairingHamiltonian:
    def __init__(self, Norb, Nocc, gval, delta_eps=1.0):
        self.Norb = Norb
        self.Nocc = Nocc
        self.delta_eps = delta_eps
        self.gval = gval
        self.basis = self.make_basis()
        self.epsilon = self.eval_epsilon()
        self.Hmat = self.eval_Hmat()

    def make_basis(self):
        self.basis = []
        for occ in combinations(range(self.Norb), self.Nocc):
            self.basis.append(occ)

        return self.basis
    
    def eval_epsilon(self):
        self.epsilon = [ 2 * i * self.delta_eps for i in range(self.Norb) ]
        return self.epsilon
    
    def eval_Hmat(self):
        dim = len(self.basis)
        self.Hmat = np.zeros((dim, dim))
        for bra_idx, bra in enumerate(self.basis):
            for ket_idx, ket in enumerate(self.basis):
                # Hamming distance
                diff = [ i for i in bra if i not in ket ]
                same = [ i for i in bra if i in ket ]
                # for SPE term
                if bra_idx == ket_idx:
                    self.Hmat[bra_idx, ket_idx] += np.sum( [self.epsilon[i] for i in same])
                    self.Hmat[bra_idx, ket_idx] += - self.gval * len(same) 
                # for pairing term
                if len(diff) == 1:
                    self.Hmat[bra_idx, ket_idx] = - self.gval

        return self.Hmat

def tuple_to_bitstring(tup, Norb, rev=True):
    bitint = 0
    for i in tup:
        bitint += 2**i
    if rev:
        bitstring = "|"+format(bitint, f'0{Norb}b')[::-1]+">"
    else:
        bitstring = "|"+format(bitint, f'0{Norb}b')+">"        
    return bitstring

def cG1(circ, c_qubit, i, j, theta):
    theta_4 = theta / 4 / (np.pi/2)
    circ.CX(i,j)
    circ.Ry(theta_4, i)
    circ.CX(j,i)
    circ.Ry(-theta_4, i)
    circ.CX(c_qubit, i)
    circ.Ry(theta_4, i)
    circ.CX(j,i)
    circ.Ry(-theta_4, i)
    circ.CX(c_qubit, i)
    circ.CX(i,j)

def G(circ, i, j, theta):
    theta_2 = theta / 2 / (np.pi/2)
    circ.CX(i,j)
    circ.Ry(theta_2, i)
    circ.CX(j,i)
    circ.Ry(-theta_2, i)
    circ.CX(j,i)
    circ.CX(i,j)  

Norb = 4
Nocc = 2
gval = 0.33  

Hamil = PairingHamiltonian(Norb, Nocc, gval)
evals, evecs = np.linalg.eigh(Hamil.Hmat)
evals = np.linalg.eigvalsh(Hamil.Hmat)
Egs_exact = evals[0]
E_HF = Hamil.Hmat[0,0]

print("basis:", Hamil.basis)
print([tuple_to_bitstring(tup, Norb) for tup in Hamil.basis])
print("eps: ", Hamil.epsilon)
print("Hmat: ", Hamil.Hmat)
print("evals: ", evals)
print("Egs_exact: ", Egs_exact, " E_HF", E_HF)
print("gs evec", evecs[:,0])
print("gs prob", evecs[:,0]**2)

params_exact = 0.5* np.array(
    [-0.48104276, -1.03976498, -0.98963981, -1.18481738, -0.54832984]
) 
# pytketの回転ゲートはpi/2単位で指定するので、pi/2で割っておく
params_exact = params_exact #

# Qubit Hamiltonian
SPEs = Hamil.epsilon
obs = [ ]
coeffs = [ ]
H_qubit = QubitOperator()

## 1-Zp term
for i in range(Hamil.Norb):
    H_qubit += (  0.5 * (SPEs[i] - Hamil.gval) * (QubitOperator('') - QubitOperator('Z'+str(i)) ) ) 
## XX+YY term proportional to g (pairing strength)
for i in range(Hamil.Norb):
    for j in range(i+1, Hamil.Norb):
        if i == j:
            continue
        factor = - Hamil.gval / 2 
        XX = QubitOperator('X'+str(i)+' X'+str(j), factor)
        YY = QubitOperator('Y'+str(i)+' Y'+str(j), factor)
        H_qubit += XX 
        H_qubit += YY

def ansatz(circ, register, params, idx_offset=0, method="FCI"):    
    circ.X(register[0])
    circ.X(register[1])
    if method == "FCI":
        G(circ, register[1], register[2], params[0])
        G(circ, register[2], register[3], params[1])
        cG1(circ, register[2], register[0], register[1], params[2])
        cG1(circ, register[3], register[0], register[1], params[3])
        cG1(circ, register[3], register[1], register[2], params[4])

pauli_sym = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

def qps_from_openfermion(paulis):
    """Convert OpenFermion tensor of Paulis to pytket QubitPauliString."""
    qlist = []
    plist = []
    for q, p in paulis:
        qlist.append(Qubit(q))
        plist.append(pauli_sym[p])
    return QubitPauliString(qlist, plist)

def qpo_from_openfermion(openf_op):
    """Convert OpenFermion QubitOperator to pytket QubitPauliOperator."""
    tk_op = dict()
    for term, coeff in openf_op.terms.items():
        string = qps_from_openfermion(term)
        tk_op[string] = coeff
    return QubitPauliOperator(tk_op)

hamiltonian_op = qpo_from_openfermion(H_qubit)
print("hamiltonian_op", hamiltonian_op)

basis: [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
['|1100>', '|1010>', '|1001>', '|0110>', '|0101>', '|0011>']
eps:  [0.0, 2.0, 4.0, 6.0]
Hmat:  [[ 1.34 -0.33 -0.33 -0.33 -0.33  0.  ]
 [-0.33  3.34 -0.33 -0.33  0.   -0.33]
 [-0.33 -0.33  5.34  0.   -0.33 -0.33]
 [-0.33 -0.33  0.    5.34 -0.33 -0.33]
 [-0.33  0.   -0.33 -0.33  7.34 -0.33]
 [ 0.   -0.33 -0.33 -0.33 -0.33  9.34]]
evals:  [1.18985184 3.29649666 5.34       5.34       7.42853393 9.44511758]
Egs_exact:  1.1898518351360725  E_HF 1.3399999999999999
gs evec [0.97121327 0.18194077 0.09817385 0.09817385 0.06360816 0.01789242]
gs prob [9.43255208e-01 3.31024447e-02 9.63810492e-03 9.63810492e-03
 4.04599822e-03 3.20138762e-04]
hamiltonian_op {(): 5.34000000000000, (Zq[0]): 0.165000000000000, (Zq[1]): -0.835000000000000, (Zq[2]): -1.83500000000000, (Zq[3]): -2.83500000000000, (Xq[0], Xq[1]): -0.165000000000000, (Yq[0], Yq[1]): -0.165000000000000, (Xq[0], Xq[2]): -0.165000000000000, (Yq[0], Yq[2]): -0.165000000000000, (Xq[0], Xq

## 制御ユニタリゲート

pytketで$\exp{(i\hat{H}t)}$などの演算を行う方法を調べたが...ない。


## Hadamard test

In [3]:
n_ancilla = 1
def make_Hadamard_circuit(n_ancilla, Norb, params, method="FCI"):
    circ = Circuit()
    ancilla_register = circ.add_q_register("m", n_ancilla)
    target_register = circ.add_q_register("p", Norb)
    print("ancilla_register", ancilla_register)
    print("target_register", target_register)
    ## Hadamard gate
    for a in ancilla_register:
        print("a",a)
        circ.H(a)
    ## State preparation
    register_p = list(target_register)
    ansatz(circ, register_p, params, idx_offset=n_ancilla, method=method)

    # controlled unitary
    # うわああああ

    # Hadamard gate for ancilla
    for a in ancilla_register:
        circ.H(a)

    # Measurement (if needed)

    return circ

params = params_exact.copy()

circ_Hadamard = make_Hadamard_circuit(n_ancilla, Norb, params)
draw(circ_Hadamard)


ancilla_register m
target_register p
a m[0]


へんじがないただのしかばねのようだ